# Methode Exacte : Branch & Bound

Nous proposons un algorithme exacte de resolution du probleme de flow shop de permutation. selon les criteres suivants

## Strategie de Recherche

L'algorithme est parametrable par la strategie de recherche . On propose 2 strategie:

    - Recherche en profondeur : Depth First Search
    - Recherche Meilleur D'abord : Best First Search
selon l'instance les 2 algoritmes peuvent donner des resultats different parfois meme tres performants. 

## Espace de recherche

![branch and bound tree](./images/bnb.png)

# Evaluation des noeuds 

** Evaluation noeud Interne ** : calculer le Makespan $C^*_{max}$ de la sous sequence scheduled de ce noeud

** Evaluation feuille ** : calculer le Makespan total $C_{max}$

# Elagage des noeud interne :

Un noeud interne est élagué et donc non exploré si est seuelement si : 
$$
C^*_{max} > UpperBound
$$

In [2]:
import sys
sys.path.append("../") # in order to import fsp
from fsp import branch_and_bound
import matplotlib.pyplot as plt
from utils import Instance, Benchmark,JsonBenchmark
import numpy as np

In [3]:
import plotly.figure_factory as ff
from datetime import datetime
import numpy as np
import plotly.express as px
instance4 = Instance(
    np.array([
        [1,2,3,2],
        [1,4,2,10],
        [3,2,1,5],
        [4,10,3,1],
        [1,5,4,4],
        [2,3,2,6],
        [5,2,1,1],
        [2,3,2,6],
        [5,2,1,1],
    ], dtype=np.int64)
)
results = branch_and_bound.get_results(instance4,search_strategy=branch_and_bound.DEPTH_FIRST_SEARCH,use_heuristique_init=True,log=False)
print(results)
res = instance4.get_chart_data(results)

fig = px.timeline(res['df'], x_start="Start", x_end="Finish", y="Task", color="Resource")
fig.layout.xaxis.update({
        'tickvals' : res['date_ticks'],
        'ticktext' : res['num_tick_labels']
        })
fig.show()

start seq(0, 6, 8, 2, 5, 7, 4, 1, 3)
upper bound 49.0
{'C_max': 42.0, 'order': [0, 2, 1, 4, 5, 8, 3, 7, 6], 'details': {'explored': 229926, 'pruned': 180532, 'leafs': 4, 'time': 5.358626100000009}}


In [5]:
# without heuristique init (uses starting sequence)
results2 = branch_and_bound.get_results(instance4,search_strategy=branch_and_bound.DEPTH_FIRST_SEARCH,use_heuristique_init=False,log=False)
print(results2)
res2 = instance4.get_chart_data(results2)

fig2 = px.timeline(res2['df'], x_start="Start", x_end="Finish", y="Task", color="Resource")
fig2.layout.xaxis.update({
        'tickvals' : res['date_ticks'],
        'ticktext' : res['num_tick_labels']
        })
fig2.show()

start seq(0, 1, 2, 3, 4, 5, 6, 7, 8)
upper bound 46.0
{'C_max': 42.0, 'order': [0, 2, 1, 4, 5, 8, 3, 7, 6], 'details': {'explored': 229925, 'pruned': 180532, 'leafs': 3, 'time': 5.519897700000001}}


In [6]:
jsonbenchmark_9_4 = JsonBenchmark(9,4,benchmark_folder="../benchmarks")
instance0 = jsonbenchmark_9_4.get_instance_by_index(0)["instance"]


# Depth-First-Search vs Best-First-Search

In [9]:
jobs = 9
machines = 4
random_mat = np.random.random((jobs,machines)) * 100
randomInstance = Instance(random_mat)
random_mat

array([[ 7.29664078, 32.79235022, 12.65248845, 70.17581821],
       [85.61358352, 78.71713412, 25.41374203,  6.21330961],
       [41.96685528, 84.97429538, 67.15504805, 45.79859315],
       [24.87315714, 95.30161787, 50.12517465, 11.87287397],
       [83.79829395, 83.52800971, 80.35527954, 34.96488017],
       [44.06140338, 74.15141104, 78.8861939 , 90.45449309],
       [ 0.16333123, 38.99019389, 79.07744204, 59.16274235],
       [16.27237175, 82.45826004, 10.11636772, 99.40155609],
       [45.07670546, 50.80566562, 33.76890767, 10.02614958]])

In [11]:
depth_res = branch_and_bound.get_results(randomInstance,search_strategy=branch_and_bound.DEPTH_FIRST_SEARCH,use_heuristique_init=False,log=False)
depth_res

start seq(0, 1, 2, 3, 4, 5, 6, 7, 8)
upper bound 853.7176959384797


{'C_max': 653.509320755687,
 'order': [6, 0, 2, 3, 5, 4, 7, 8, 1],
 'details': {'explored': 222228,
  'pruned': 213092,
  'leafs': 29,
  'time': 5.104607500000014}}

In [13]:
best_res = branch_and_bound.get_results(randomInstance,search_strategy=branch_and_bound.BEST_FIRST_SEARCH,use_heuristique_init=False,log=False)
best_res

start seq(0, 1, 2, 3, 4, 5, 6, 7, 8)
upper bound 217.585091597295


{'C_max': 166.8449145931314,
 'order': [1, 5, 4, 3, 7, 2, 0, 8, 6],
 'details': {'explored': 130820,
  'pruned': 167835,
  'leafs': 14,
  'time': 3.572050900000022}}

# Impact de la distribution initiale des couts sur les machines

## Instance a couts suivant une loi normale 

In [15]:
jobs = 9
machines = 3
mean_time = 10
std_time = 20
random_mat = np.abs(np.random.normal(loc=mean_time,scale=std_time,size=(jobs,machines)))
randomInstance = Instance(random_mat)
random_mat

array([[2.07647373e+01, 4.40797413e+01, 3.51989113e+01],
       [2.29571313e+01, 1.87079955e+01, 1.42286374e+01],
       [9.24237116e+00, 5.52424480e+00, 2.03434826e+01],
       [7.88994235e+00, 1.29964754e+01, 7.70927630e+00],
       [2.02476758e-02, 1.86682329e+01, 1.93785638e+01],
       [1.07756875e+01, 6.13715381e+00, 2.23989700e+01],
       [4.88220069e+01, 7.43746254e-01, 6.48838971e+00],
       [6.74721000e+00, 5.26542607e+00, 5.93438712e+00],
       [3.35448591e+01, 3.44490756e+01, 3.22614494e+01]])

In [17]:
heur_res = branch_and_bound.get_results(randomInstance,search_strategy=branch_and_bound.DEPTH_FIRST_SEARCH,use_heuristique_init=True,log=False)
heur_res

start seq(7, 3, 2, 4, 5, 1, 6, 0, 8)
upper bound 238.75943625419586


{'C_max': 186.70383618853378,
 'order': [2, 4, 5, 0, 7, 8, 1, 3, 6],
 'details': {'explored': 187742,
  'pruned': 136821,
  'leafs': 15,
  'time': 3.1009382000000016}}

In [19]:
noheur_res = branch_and_bound.get_results(randomInstance,search_strategy=branch_and_bound.DEPTH_FIRST_SEARCH,use_heuristique_init=False,log=False)
noheur_res

start seq(0, 1, 2, 3, 4, 5, 6, 7, 8)
upper bound 897.5336277667755


{'C_max': 680.5297917460082,
 'order': [2, 3, 5, 7, 8, 1, 4, 6, 0],
 'details': {'explored': 153725,
  'pruned': 216449,
  'leafs': 31,
  'time': 3.601382199999989}}

## Instance a couts suivant une loi uniforme

In [21]:
jobs = 9
machines = 4
low = 10
high = 100
random_mat = np.abs(np.random.uniform(low=low,high=high,size=(jobs,machines)))
randomInstance = Instance(random_mat)
random_mat

array([[32.47570855, 86.27783664, 21.00178795, 23.90687574],
       [36.70168868, 87.06616481, 25.77035566, 60.30412918],
       [59.98136318, 78.98230927, 39.21030938, 12.69193125],
       [46.66738197, 73.45435191, 29.42812288, 49.2310704 ],
       [56.18083101, 24.48163612, 34.95014082, 11.01029671],
       [86.96912203, 65.53020877, 81.97162692, 90.97316585],
       [20.95129667, 43.106322  , 24.25951603, 66.98832381],
       [61.68388203, 74.27892626, 77.29162074, 14.83115469],
       [80.2619937 , 11.62061975, 22.4876214 , 70.09787929]])

In [23]:
heur_res = branch_and_bound.get_results(randomInstance,search_strategy=branch_and_bound.DEPTH_FIRST_SEARCH,use_heuristique_init=True,log=False)
heur_res

start seq(4, 6, 0, 8, 2, 3, 1, 7, 5)
upper bound 813.8693588756914


{'C_max': 611.710109739686,
 'order': [6, 1, 5, 3, 7, 2, 8, 0, 4],
 'details': {'explored': 203131,
  'pruned': 227135,
  'leafs': 20,
  'time': 4.378471399999995}}

In [25]:
noheur_res = branch_and_bound.get_results(randomInstance,search_strategy=branch_and_bound.DEPTH_FIRST_SEARCH,use_heuristique_init=False,log=False)
noheur_res

start seq(0, 1, 2, 3, 4, 5, 6, 7, 8)
upper bound 226.3619267258435


{'C_max': 174.35212423973746,
 'order': [6, 3, 8, 4, 2, 7, 0, 1, 5],
 'details': {'explored': 159613,
  'pruned': 148847,
  'leafs': 26,
  'time': 3.437460200000004}}

## Instance a cout de loi exponentielle

In [27]:
jobs = 9
machines = 4
scale = 10
random_mat = np.random.exponential(scale=scale,size=(jobs,machines))
randomInstance = Instance(random_mat)
random_mat

array([[ 6.99053912, 14.45032559, 20.24925349, 10.89070226],
       [ 0.06102596,  5.93200728,  0.43653829,  6.81352417],
       [ 8.387636  , 23.27849514, 13.21860342, 12.72384072],
       [19.94169546,  1.52236559, 13.28755226,  5.81094275],
       [ 5.30026155, 37.89829988,  6.64273226,  3.42433932],
       [ 1.27657261,  0.89839313,  0.2940803 ,  3.13231312],
       [ 5.17817005, 27.54513524,  0.12961476,  7.46833826],
       [ 0.72454487, 11.0737484 , 17.40085571,  6.26392235],
       [28.06587982, 21.24261877,  8.72730737, 20.453908  ]])

In [29]:
heur_res = branch_and_bound.get_results(randomInstance,search_strategy=branch_and_bound.DEPTH_FIRST_SEARCH,use_heuristique_init=True,log=False)
heur_res

start seq(5, 1, 7, 6, 3, 0, 4, 2, 8)
upper bound 174.2991769958119


{'C_max': 151.50036798394288,
 'order': [1, 4, 0, 5, 8, 3, 2, 7, 6],
 'details': {'explored': 245088,
  'pruned': 184862,
  'leafs': 6,
  'time': 4.906749200000007}}

In [30]:
noheur_res = branch_and_bound.get_results(randomInstance,search_strategy=branch_and_bound.DEPTH_FIRST_SEARCH,use_heuristique_init=False,log=False)
noheur_res

start seq(0, 1, 2, 3, 4, 5, 6, 7, 8)
upper bound 180.01314350713866


{'C_max': 151.50036798394288,
 'order': [1, 4, 0, 5, 8, 3, 2, 7, 6],
 'details': {'explored': 245089,
  'pruned': 184862,
  'leafs': 7,
  'time': 4.691052899999988}}